### Código para automatizar envio de E-mails

In [4]:
import pandas as pd
import smtplib
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def ler_emails(planilha_path):
    # Lê a coluna 'E-mail' da planilha
    df_emails = pd.read_excel(planilha_path)
    if 'E-mail' in df_emails.columns:
        tamanho_antes = len(df_emails)
        
        # Remove duplicatas com base na coluna 'E-mail'
        df_sem_duplicatas = df_emails.drop_duplicates(
            subset=['E-mail'], keep='first')
        
        tamanho_depois = len(df_sem_duplicatas)
        
        if tamanho_antes != tamanho_depois:
            print(f"\nE-mails duplicados foram removidos! Agora são: {tamanho_depois}\n")
        else:
            emails = df_emails['E-mail'].tolist()
            
            print('Não há E-mails duplicados na planilha!')
            
            return emails, df_emails

        df_sem_duplicatas.to_excel('Enviar E-mails.xlsx', index=False)

        # Retorna a lista de e-mails e o DataFrame sem duplicatas
        return df_sem_duplicatas['E-mail'].tolist(), df_sem_duplicatas


def enviar_email(destinatarios, planilha_path):
    # Configurações para o servidor SMTP do Outlook
    servidor_smtp = 'smtp.office365.com'
    porta_smtp = 587
    
    # Configurando email do remetente e senha
    email_remetente = 'oversouls11@gmail.com'
    senha = 'darksouls860'

    # Configuração inicial da mensagem comum a todos os e-mails
    corpo_email = """
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 20px;
            }
            h1 {
                color: #333;
            }
            p {
                color: #555;
            }
        </style>
    </head>
    <body>
        <h1>Teste de E-mail</h1>
        <p>Olá,</p>
    </body>
    </html>
    """
    
    df_emails = pd.read_excel(planilha_path)
    status_coluna = 'STATUS'

    for email_destino in destinatarios:
        # Cria uma nova mensagem para cada e-mail
        msg = MIMEMultipart()
        
        # Adiciona o corpo do e-mail
        corpo_mensagem = MIMEText(corpo_email, 'html')
        msg.attach(corpo_mensagem)

        # Configura o destinatário atual
        msg['To'] = email_destino
        msg['Subject'] = "Um teste de envio de e-mail"  # Assunto do email
        msg['From'] = email_remetente # Substitua pelo seu endereço de e-mail do Outlook
        password = senha  # Substitua pela sua senha ou pela senha de aplicativo

        try:
            # Inicia a conexão SMTP e envia o e-mail
            with smtplib.SMTP(servidor_smtp, porta_smtp) as s:
                s.starttls()
                # Login Credentials for sending the mail
                s.login(msg['From'], password)
                s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
            
            # Atualiza o status para 'SUCESSO'    
            df_emails.loc[df_emails['E-mail'] == email_destino, status_coluna] = 'SUCESSO'
            print(f'E-mail enviado para {email_destino}')
            
        except Exception as e:
            # Se ocorrer um erro, imprima a mensagem de erro e atualize o status para 'ERRO'
            print(f'Erro ao enviar e-mail para {email_destino}: {e}')
            df_emails.loc[df_emails['E-mail'] == email_destino, status_coluna] = 'ERRO'

# Caminho da planilha com os e-mails
planilha_path = Path('Enviar E-mails.xlsx')

# Lê os e-mails da planilha
destinatarios, _ = ler_emails(planilha_path)

print('Iniciando envio de e-mails!')

# Envia e-mails para os destinatários
enviar_email(destinatarios, planilha_path)

print('Todos os e-mails enviados com sucesso!')


E-mails duplicados foram removidos! Agora são: 3

Iniciando envio de e-mails!
E-mail enviado para emailqualquer1@gmail.com
E-mail enviado para emailqualquer2@gmail.com
E-mail enviado para emailqualquer3@gmail.com
Todos os e-mails enviados com sucesso!
